In [1]:
dataset = "mimic3"
task = "mortality"
kg = "GPT-KG"

In [2]:
from graphcare import *

sample_dataset, G, ent2id, rel2id, ent_emb, rel_emb, \
            map_cluster, map_cluster_inv, map_cluster_rel, map_cluster_rel_inv, \
                ccscm_id2clus, ccsproc_id2clus, atc3_id2clus = load_everything(dataset, task, kg, kg_ratio=1.0)

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
mode, out_channels, loss_function = get_mode_and_out_channels_and_loss_func(task=task, sample_dataset=sample_dataset)

# label direct ehr node
print("Labeling direct ehr nodes...")
sample_dataset = label_ehr_nodes(task, sample_dataset, len(map_cluster), ccscm_id2clus, ccsproc_id2clus, atc3_id2clus)
# sample_dataset = label_k_hop_nodes(G=G_tg, dataset=sample_dataset, k=2)
print("Splitting dataset...")
# train_dataset, val_dataset, test_dataset = split_by_patient(sample_dataset, [0.8, 0.1, 0.1], train_ratio=1.0, seed=528)

Labeling direct ehr nodes...


100%|██████████| 9717/9717 [00:00<00:00, 13878.19it/s]

Splitting dataset...


In [4]:
ehr_nodes = set(ccscm_id2clus.values()).union(set(ccsproc_id2clus.values())).union(set(atc3_id2clus.values()))

In [5]:
import random

ratio = 1.0
node_frac = 1- ratio

nodes = list(G.nodes())
random.shuffle(nodes)

num_nodes_to_remove = int(node_frac * G.number_of_nodes())
nodes_to_remove = nodes[:num_nodes_to_remove]

for node in nodes_to_remove:
    if str(node) not in ehr_nodes:
        G.remove_node(node)
G_tg = from_networkx(G)

In [6]:
y_to_x_map = {}
for i, y in enumerate(G_tg.y):
    y_to_x_map[int(y)] = i

In [7]:
from copy import deepcopy

for patient in tqdm(sample_dataset):
    node_list = []
    for node in patient['node_set']:
        if node not in nodes_to_remove or str(node) in ehr_nodes:
            node_list.append(node)

    patient['node_set'] = [y_to_x_map[y] for y in node_list]

100%|██████████| 9717/9717 [00:02<00:00, 3987.44it/s]


In [8]:
all_set = Dataset(G=G_tg, dataset=sample_dataset, task=task, strategy="1")

In [9]:
from tqdm import tqdm

nodes_count = 0
edges_count = 0
cnt = 0

for i in tqdm(range(len(all_set))):
    try:
        nodes_count += all_set[i].x.shape[0]
        edges_count += all_set[i].edge_index.shape[1]
        cnt += 1
    except:
        continue

avg_nodes = nodes_count / cnt
avg_edges = edges_count / cnt
print("avg_nodes: ", avg_nodes)
print("avg_edges: ", avg_edges)

 34%|███▍      | 3337/9717 [00:32<02:48, 37.81it/s] 

In [ ]:
for patient in sample_dataset:
    patient['node_set'] = []

In [ ]:
sample_dataset = label_ehr_nodes(task, sample_dataset, len(map_cluster), ccscm_id2clus, ccsproc_id2clus, atc3_id2clus)


100%|██████████| 9717/9717 [00:01<00:00, 7864.84it/s]


In [ ]:
nodes_count = 0
edges_count = 0
cnt = 0


for patient in sample_dataset:
    nodes_count += len(patient['node_set'])
    edges_count += len(patient['edge_set'])
    cnt += 1

avg_nodes = nodes_count / cnt
avg_edges = edges_count / cnt
print("avg_nodes: ", avg_nodes)
print("avg_edges: ", avg_edges)

KeyError: 'edge_set'